## Project Parameters

In [1]:
project_name = "k9mail-5.304"
project_root = "Datasets/k9mail/"
src_path = "k9mail-5.304/k9mail/src/main/java/"
graph_path = "k9mail-5.304.json"

(project_name,project_root,src_path,graph_path)

('k9mail-5.304',
 'Datasets/k9mail/',
 'k9mail-5.304/k9mail/src/main/java/',
 'k9mail-5.304.json')

## Imports

In [2]:
import random
import json
import networkx as nx

## Relationship Operations

In [3]:
def transitive_closure(tuples1, tuples2):
    result = set()

    # Create a dictionary to store intermediate values
    intermediate_dict = {}

    # Populate the intermediate dictionary with values from list2
    for b1, c1 in tuples2:
        if b1 not in intermediate_dict:
            intermediate_dict[b1] = [c1]
        else:
            intermediate_dict[b1].append(c1)

    # Iterate through list1 and find transitive pairs
    for a, b in tuples1:
        if b in intermediate_dict:
            for c in intermediate_dict[b]:
                result.add((a, c))

    return result

# Example usage:
t1 = [(1, 2), (2, 3), (3, 4)]
t2 = [(2, 'a'), (3, 'b'), (4, 'c'), (4, 'd'), (3, 'c')]

result = transitive_closure(t1, t2)
print(result)


{(3, 'd'), (2, 'b'), (2, 'c'), (1, 'a'), (3, 'c')}


In [4]:
def flip_tuples(tuples):
    return {(b, a) for a, b in tuples}

# Example usage:
t = [(1, 2), (3, 4), ('a', 'b')]
flipped = flip_tuples(t)

print(flipped)


{('b', 'a'), (2, 1), (4, 3)}


## Load javapers output

In [5]:
with open(project_root+graph_path, encoding="utf8") as json_file:
    data = json.load(json_file)

data['elements']['nodes'][0], data['elements']['edges'][0]

({'data': {'id': 'com.fsck.k9.activity.compose.RecipientLoader.PROJECTION_CRYPTO_ADDRESSES',
   'properties': {'sourceText': 'private static final java.lang.String[] PROJECTION_CRYPTO_ADDRESSES = new java.lang.String[]{ "address", "uid_address" };',
    'visibility': 'private',
    'simpleName': 'PROJECTION_CRYPTO_ADDRESSES',
    'kind': 'field',
    'metaSrc': 'source code'},
   'labels': ['Variable']}},
 {'data': {'id': 'ac7cba3e978bef0c69a74a9b1b4a1e08',
   'source': 'com.fsck.k9.notification.NotificationController.getAccountName(com.fsck.k9.Account).account',
   'label': 'type',
   'properties': {'weight': 1, 'metaSrc': 'source code'},
   'target': 'com.fsck.k9.Account'}})

In [6]:
nodes = {(node['data']['id'],node['data']['labels'][0]):node['data']['properties'] for node in data['elements']['nodes']}
edges = {(edge['data']['source'],edge['data']['target'],edge['data']['label']):edge['data']['properties'] for edge in data['elements']['edges']}

(len(nodes), len(edges)), random.sample(nodes.items(), 5), random.sample(edges.items(), 5)

((13432, 28061),
 [(('com.fsck.k9.mailstore.LockableDatabase$SchemaDefinition.doDbUpgrade(android.database.sqlite.SQLiteDatabase).db',
    'Variable'),
   {'simpleName': 'db', 'kind': 'parameter', 'metaSrc': 'source code'}),
  (('com.fsck.k9.provider.EmailProvider$StatsColumns', 'Structure'),
   {'docComment': '',
    'simpleName': 'StatsColumns',
    'kind': 'interface',
    'metaSrc': 'source code'}),
  (('com.fsck.k9.widget.list.MessageListWidgetProvider.composeActionPendingIntent(android.content.Context)',
    'Operation'),
   {'sourceText': 'private android.app.PendingIntent composeActionPendingIntent(android.content.Context context) {\r\n    android.content.Intent intent = new android.content.Intent(context, com.fsck.k9.activity.MessageCompose.class);\r\n    intent.setAction(com.fsck.k9.activity.MessageCompose.ACTION_COMPOSE);\r\n    return android.app.PendingIntent.getActivity(context, 0, intent, PendingIntent.FLAG_UPDATE_CURRENT);\r\n}',
    'docComment': '',
    'visibility': 

In [8]:
rel_types = {typ for (_, _, typ) in edges}
rel_types

{'contains',
 'hasParameter',
 'hasScript',
 'hasVariable',
 'instantiates',
 'invokes',
 'returnType',
 'specializes',
 'type'}

In [9]:
detailed_rels = {rel_type:{(source,target) for (source, target, typ) in edges if typ == rel_type} for rel_type in rel_types}
detailed_rels

{'hasScript': {('com.fsck.k9.activity.compose.RecipientMvpView',
   'com.fsck.k9.activity.compose.RecipientMvpView.showCcUncompletedError()'),
  ('com.fsck.k9.activity.MessageLoaderHelper$MessageLoaderCallbacks',
   'com.fsck.k9.activity.MessageLoaderHelper$MessageLoaderCallbacks.onMessageViewInfoLoadFinished(com.fsck.k9.mailstore.MessageViewInfo)'),
  ('com.fsck.k9.activity.setup.AccountSetupCheckSettings',
   'com.fsck.k9.activity.setup.AccountSetupCheckSettings.handleCertificateValidationException(com.fsck.k9.mail.CertificateValidationException)'),
  ('com.fsck.k9.activity.compose.ComposeCryptoStatus',
   'com.fsck.k9.activity.compose.ComposeCryptoStatus.getOpenPgpKeyId()'),
  ('com.fsck.k9.activity.MessageList',
   'com.fsck.k9.activity.MessageList.setProgress(boolean)'),
  ('com.fsck.k9.activity.compose.RecipientPresenter',
   'com.fsck.k9.activity.compose.RecipientPresenter.onMenuSetSignOnly(boolean)'),
  ('com.fsck.k9.K9', 'com.fsck.k9.K9.showContactName()'),
  ('com.fsck.k9.act

In [10]:
containers = {id for (id,label) in nodes if label == 'Container'}
structures = {id for (id,label) in nodes if label == 'Structure'}
containers, structures

({'com',
  'com.fsck',
  'com.fsck.k9',
  'com.fsck.k9.account',
  'com.fsck.k9.activity',
  'com.fsck.k9.activity.compose',
  'com.fsck.k9.activity.loader',
  'com.fsck.k9.activity.misc',
  'com.fsck.k9.activity.setup',
  'com.fsck.k9.autocrypt',
  'com.fsck.k9.cache',
  'com.fsck.k9.controller',
  'com.fsck.k9.crypto',
  'com.fsck.k9.fragment',
  'com.fsck.k9.helper',
  'com.fsck.k9.helper.jsoup',
  'com.fsck.k9.mailstore',
  'com.fsck.k9.mailstore.migrations',
  'com.fsck.k9.mailstore.util',
  'com.fsck.k9.message',
  'com.fsck.k9.message.extractors',
  'com.fsck.k9.message.html',
  'com.fsck.k9.message.quote',
  'com.fsck.k9.message.signature',
  'com.fsck.k9.notification',
  'com.fsck.k9.power',
  'com.fsck.k9.preferences',
  'com.fsck.k9.provider',
  'com.fsck.k9.remotecontrol',
  'com.fsck.k9.search',
  'com.fsck.k9.service',
  'com.fsck.k9.setup',
  'com.fsck.k9.ui',
  'com.fsck.k9.ui.compose',
  'com.fsck.k9.ui.crypto',
  'com.fsck.k9.ui.dialog',
  'com.fsck.k9.ui.message',
  

In [11]:
abstract_rels = dict()

abstract_rels['contains'] = detailed_rels['contains']
abstract_rels['specializes'] = detailed_rels['specializes']
abstract_rels['holds'] = transitive_closure(detailed_rels['hasVariable'], detailed_rels['type'])
abstract_rels['accepts'] = transitive_closure(transitive_closure(detailed_rels['hasScript'], detailed_rels['hasParameter']), detailed_rels['type'])
abstract_rels['returns'] = transitive_closure(detailed_rels['hasScript'], detailed_rels['returnType'])
abstract_rels['constructs'] = transitive_closure(detailed_rels['hasScript'], detailed_rels['instantiates'])
abstract_rels['calls'] = transitive_closure(transitive_closure(detailed_rels['hasScript'], detailed_rels['invokes']), flip_tuples(detailed_rels['hasScript']))

[(rel_type, len(abstract_rels[rel_type])) for rel_type in abstract_rels]

[('contains', 925),
 ('specializes', 159),
 ('holds', 848),
 ('accepts', 1107),
 ('returns', 1129),
 ('constructs', 502),
 ('calls', 1351)]

In [12]:
abstract_rels

{'contains': {('com', 'com.fsck'),
  ('com.fsck', 'com.fsck.k9'),
  ('com.fsck.k9', 'com.fsck.k9.Account'),
  ('com.fsck.k9', 'com.fsck.k9.Account$DeletePolicy'),
  ('com.fsck.k9', 'com.fsck.k9.Account$Expunge'),
  ('com.fsck.k9', 'com.fsck.k9.Account$FolderMode'),
  ('com.fsck.k9', 'com.fsck.k9.Account$MessageFormat'),
  ('com.fsck.k9', 'com.fsck.k9.Account$QuoteStyle'),
  ('com.fsck.k9', 'com.fsck.k9.Account$Searchable'),
  ('com.fsck.k9', 'com.fsck.k9.Account$ShowPictures'),
  ('com.fsck.k9', 'com.fsck.k9.Account$SortType'),
  ('com.fsck.k9', 'com.fsck.k9.AccountStats'),
  ('com.fsck.k9', 'com.fsck.k9.BaseAccount'),
  ('com.fsck.k9', 'com.fsck.k9.Clock'),
  ('com.fsck.k9', 'com.fsck.k9.EmailAddressValidator'),
  ('com.fsck.k9', 'com.fsck.k9.FontSizes'),
  ('com.fsck.k9', 'com.fsck.k9.Globals'),
  ('com.fsck.k9', 'com.fsck.k9.Identity'),
  ('com.fsck.k9', 'com.fsck.k9.K9'),
  ('com.fsck.k9', 'com.fsck.k9.K9$ApplicationAware'),
  ('com.fsck.k9', 'com.fsck.k9.K9$BACKGROUND_OPS'),
  ('c

In [13]:
# example: lift calls

lifted_calls = transitive_closure(
	transitive_closure(
		abstract_rels['contains'], 
		abstract_rels['calls']), 
	flip_tuples(abstract_rels['contains']))
lifted_calls

{('com.fsck.k9', 'com.fsck.k9'),
 ('com.fsck.k9', 'com.fsck.k9.Account'),
 ('com.fsck.k9', 'com.fsck.k9.K9'),
 ('com.fsck.k9', 'com.fsck.k9.PRNGFixes'),
 ('com.fsck.k9', 'com.fsck.k9.Throttle'),
 ('com.fsck.k9', 'com.fsck.k9.controller'),
 ('com.fsck.k9', 'com.fsck.k9.helper'),
 ('com.fsck.k9', 'com.fsck.k9.mailstore'),
 ('com.fsck.k9', 'com.fsck.k9.power'),
 ('com.fsck.k9', 'com.fsck.k9.preferences'),
 ('com.fsck.k9', 'com.fsck.k9.preferences.GlobalSettings'),
 ('com.fsck.k9', 'com.fsck.k9.provider'),
 ('com.fsck.k9', 'com.fsck.k9.search'),
 ('com.fsck.k9', 'com.fsck.k9.service'),
 ('com.fsck.k9', 'com.fsck.k9.widget.list'),
 ('com.fsck.k9.PRNGFixes', 'com.fsck.k9'),
 ('com.fsck.k9.PRNGFixes', 'com.fsck.k9.PRNGFixes'),
 ('com.fsck.k9.activity', 'com.fsck.k9'),
 ('com.fsck.k9.activity', 'com.fsck.k9.activity'),
 ('com.fsck.k9.activity', 'com.fsck.k9.activity.Accounts'),
 ('com.fsck.k9.activity', 'com.fsck.k9.activity.AlternateRecipientAdapter'),
 ('com.fsck.k9.activity', 'com.fsck.k9.a

In [7]:
def find_nodes(nodes, id):
	return [(id_,label) for (id_,label) in nodes if id_ == id]

## networkx Stuff

In [14]:
g = nx.MultiDiGraph()

for (id, label), properties in nodes.items():
    g.add_node(id, label=label, **properties)

for (source, target, typ), properties in edges.items():
    g.add_edge(source, target, type=typ, **properties)

g.number_of_nodes(), g.number_of_edges()

(13432, 28061)

In [15]:
for node, nbrsdict in list(g.adj.items())[:10]:
    print((node, nbrsdict))
    

('com.fsck.k9.activity.compose.RecipientLoader.PROJECTION_CRYPTO_ADDRESSES', AdjacencyView({}))
('com.fsck.k9.activity.ActivityListener.pendingCommandsProcessing(com.fsck.k9.Account).account', AdjacencyView({'com.fsck.k9.Account': {0: {'type': 'type', 'weight': 1, 'metaSrc': 'source code'}}}))
('com.fsck.k9.provider.DecryptedFileProvider.getUriForProvidedFile(android.content.Context,java.io.File,java.lang.String,java.lang.String).file', AdjacencyView({}))
('com.fsck.k9.view.ToolableViewAnimator.com.fsck.k9.view.ToolableViewAnimator(android.content.Context,android.util.AttributeSet,int).context', AdjacencyView({}))
('com.fsck.k9.activity.Accounts$ImportSelectionDialog.show(com.fsck.k9.activity.Accounts).activity', AdjacencyView({'com.fsck.k9.activity.Accounts': {0: {'type': 'type', 'weight': 1, 'metaSrc': 'source code'}}}))
('com.fsck.k9.view.CryptoModeSelector$CryptoModeSelectorState.SIGN_ONLY', AdjacencyView({'com.fsck.k9.view.CryptoModeSelector$CryptoModeSelectorState': {0: {'type': 

In [16]:
method_sources = {id:properties['sourceText'] for (id,label),properties in nodes.items() if label in ['Operation', 'Constructor']}
method_sources

{'com.fsck.k9.fragment.MessageListFragmentComparators$DateComparator.com.fsck.k9.fragment.MessageListFragmentComparators$DateComparator()': 'DateComparator() {\r\n}',
 'com.fsck.k9.notification.WearNotifications.buildStackedNotification(com.fsck.k9.Account,com.fsck.k9.notification.NotificationHolder)': 'public android.app.Notification buildStackedNotification(com.fsck.k9.Account account, com.fsck.k9.notification.NotificationHolder holder) {\r\n    int notificationId = holder.notificationId;\r\n    com.fsck.k9.notification.NotificationContent content = holder.content;\r\n    android.support.v4.app.NotificationCompat.Builder builder = createBigTextStyleNotification(account, holder, notificationId);\r\n    android.app.PendingIntent deletePendingIntent = actionCreator.createDismissMessagePendingIntent(context, content.messageReference, holder.notificationId);\r\n    builder.setDeleteIntent(deletePendingIntent);\r\n    addActions(builder, account, holder);\r\n    return builder.build();\r\n

## LLM Stuff

In [ ]:
from txtai.pipeline import LLM

# llm = LLM("Open-Orca/Mistral-7B-OpenOrca")
llm = LLM("/home/satrio/Models/microsoft/phi-2", trust_remote_code='True')
# llm = LLM("lmsys/vicuna-7b-v1.5")
for method_name, method_src in random.sample(method_sources.items(), 10):
	llm(
	f"""
	Answer the following question using the provided context.

	Question:
	What does this Java method do?

	Context:
	{method_src}
	"""
	)